<a href="https://colab.research.google.com/github/truongthuanr/self-project/blob/dev/11_Named_Entity_Recognition/Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

%%capture
!pip3 install datasets


In [4]:

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from datasets import load_dataset
import os


In [5]:
from keras.models import Model
from keras.layers import Input,Embedding,TimeDistributed,\
                         Dropout,Conv1D,MaxPooling1D,\
                         Flatten,Bidirectional,LSTM,Dense,\
                         concatenate

from keras.initializers import RandomUniform

from keras.optimizers import Adam





In [7]:
# For vetorization
max_word_tokens = 24000
max_sentence_length = 30
max_word_len = 20


# Dataset

Used dataset:

https://huggingface.co/datasets/conll2003

In [8]:
conll_data = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
type(conll_data['train']['tokens'][0])

list

In [10]:
conll_data['train']['ner_tags'][0]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

## Embedding

In [11]:
# '''
# def export_to_file(export_file_path, data):
#     with open(export_file_path, "w") as f:
#         for record in data:
#             ner_tags = record["ner_tags"]
#             tokens = record["tokens"]
#             if len(tokens) > 0:
#                 f.write(
#                     str(len(tokens))
#                     + "\t"
#                     + "\t".join(tokens)
#                     + "\t"
#                     + "\t".join(map(str, ner_tags))
#                     + "\n"
#                 )


# os.mkdir("data")
# export_to_file("./data/conll_train.txt", conll_data["train"])
# export_to_file("./data/conll_val.txt", conll_data["validation"])
# '''

# Data preprocessing


In [12]:
char2Idx = {"PADDING":0, "UNKNOWN":1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|":
    char2Idx[c] = len(char2Idx)
len(char2Idx)

95

In [13]:
def character_vectorize(X):
  data_vec = []
  for sentence in X:
    # print(f"Sentence: {sentence}")
    padchar = char2Idx['PADDING']
    sentence_vec=[]
    for word in sentence:
      # print(word)
      chars = []
      if len(word) >= max_word_len:

        chars=[char2Idx[c] for c in word[:max_word_len]]
      else:
        prepad=int((max_word_len-len(word))/2)
        postpad=max_word_len-(len(word)+prepad)
        chars.extend([padchar]*prepad)
        chars.extend([char2Idx[c] for c in word])
        chars.extend([padchar]*postpad)
      sentence_vec.append(chars)

    data_vec.append(sentence_vec)
  data_vec = np.asarray(data_vec, dtype=object)
  return data_vec




X_train_char = conll_data['train']['tokens']

X_train_char = character_vectorize(X_train_char)
X_train_char = pad_sequences(sequences = X_train_char,
                             maxlen=max_sentence_length,
                             dtype=object,
                             padding="post",
                             truncating="post",
                             value=0)
X_train_char = np.asarray(X_train_char,
                          dtype=np.float32)
X_train_char[0]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 43., 59.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 30., 17., 22., 17., 15., 32., 31.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 45., 17., 30., 25., 13., 26.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 15., 13., 24., 24.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 32., 27.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 14., 27., 37., 15., 27., 32., 32.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 40., 30., 21., 32., 21., 31., 20.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 24., 13., 25., 14.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 65.,  0.,  

In [14]:
X_train_word = np.array([ " ".join(row) for row in conll_data['train']['tokens']])
# X_train = conll_data['train']['tokens']
X_train_word

array(['EU rejects German call to boycott British lamb .',
       'Peter Blackburn', 'BRUSSELS 1996-08-22', ...,
       'Plymouth 2 Preston 1', 'Division three', 'Swansea 1 Lincoln 2'],
      dtype='<U512')

In [15]:
char2Idx = {"PADDING":0, "UNKNOWN":1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|":
    char2Idx[c] = len(char2Idx)
len(char2Idx)

95

In [19]:
label2id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
id2label = {v: k for k, v in label2id.items()}


In [20]:
y_train = pad_sequences(conll_data['train']['ner_tags'],
                        dtype='int32',
                        maxlen=max_sentence_length,
                        padding='post',
                        truncating='post',
                        value=0,)

In [21]:
y_train

array([[3, 0, 7, ..., 0, 0, 0],
       [1, 2, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       ...,
       [3, 0, 3, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 3, ..., 0, 0, 0]], dtype=int32)

In [26]:
for i, value in enumerate([11,12]):
  print(i)
  print(f"{value=}")

0
value=11
1
value=12


In [31]:
def encoding_ytrain(y_train):
  n=len(label2id)
  nrow=len(y_train)
  encoded = np.zeros(shape=(nrow,max_sentence_length,n))
  for iy, y in enumerate(y_train):
    senvec = np.zeros(shape=(max_sentence_length,n))
    for ichar, char in enumerate(y):
      charvec = np.zeros(n)
      charvec[char] = 1
      # print(f"{char=}")
      # print(f"{charvec=}")
      senvec[ichar]=charvec

    encoded[iy] = senvec
  return encoded

y_train = encoding_ytrain(y_train)


# Model

In [32]:
from keras.layers import TextVectorization

In [33]:
# Configuration



In [73]:
text_vectorizer = TextVectorization(max_tokens=max_word_tokens,   # how many word in the vocab lib
                                    standardize='lower_and_strip_punctuation', # process the tab
                                    split='whitespace', # split the token
                                    ngrams=None,
                                    output_mode='int', # map token to number
                                    output_sequence_length=max_sentence_length,
                                    pad_to_max_tokens=False,
                                    vocabulary=None,)
text_vectorizer.adapt(np.array([ " ".join(row) for row in conll_data['train']['tokens']]))

In [74]:
print(text_vectorizer.vocabulary_size())
print(text_vectorizer.get_vocabulary()[:10])

20070
['', '[UNK]', 'the', 'of', 'in', 'to', 'a', 'and', 'on', 'said']


In [75]:
char2Idx = {"PADDING":0, "UNKNOWN":1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|":
    char2Idx[c] = len(char2Idx)


len(char2Idx)

95

In [76]:
## Embeding




In [77]:
# Word level
word_input = Input(shape=(1,),dtype=tf.string,name='word_input')
# Tokenize
words_vectorize = text_vectorizer(word_input)

# Embedding
embedding = Embedding(input_dim=max_word_tokens,
                      output_dim=128,
                      embeddings_initializer="uniform",
                      input_length=None)
words = embedding(words_vectorize)


In [78]:
words

<KerasTensor: shape=(None, 30, 128) dtype=float32 (created by layer 'embedding_10')>

In [79]:
# Character level
character_input=Input(shape=(None,max_word_len,),
                      name='char_input',
                      dtype=tf.int32)
embed_char_out=TimeDistributed(Embedding(input_dim = len(char2Idx),
                                         output_dim=10,
                                         embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5),
                                         input_length=max_sentence_length,),
                                         name='char_embedding')(character_input)
dropout = Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3,
                                   filters=10,
                                   padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(max_word_len))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)

In [80]:
print(character_input.shape)
print(embed_char_out.shape)
print(dropout.shape)
print(conv1d_out.shape)
print(maxpool_out.shape)
print(char.shape)

(None, None, 20)
(None, None, 20, 10)
(None, None, 20, 10)
(None, None, 20, 10)
(None, None, 1, 10)
(None, None, 10)


In [81]:

combined = concatenate([words, char])
# combined = words
# Bi-LSTM
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(combined)

# # Output -> Tag Scoring -> Tag
output = TimeDistributed(Dense(len(label2id), activation='softmax'))(output)
model = Model(inputs=[word_input, character_input], outputs=[output])


In [82]:
model.compile(loss="binary_crossentropy",
              optimizer=Adam(),
              metrics=["accuracy"])

In [83]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 char_input (InputLayer)     [(None, None, 20)]           0         []                            
                                                                                                  
 char_embedding (TimeDistri  (None, None, 20, 10)         950       ['char_input[0][0]']          
 buted)                                                                                           
                                                                                                  
 dropout_8 (Dropout)         (None, None, 20, 10)         0         ['char_embedding[0][0]']      
                                                                                                  
 time_distributed_15 (TimeD  (None, None, 20, 10)         310       ['dropout_8[0][0]']     

In [84]:
# Charater level

# Character Embedding

# Additional character feature

# Convolution

# Max

# CNN extracted character feature


In [85]:
# Word level
# words_input = Input(shape=(None,),dtype='int32',name='words_input')
# words = Embedding(input_dim=20000,
#                   output_dim=128,
#                   trainable=False)(words_input)

# Casing
# casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
# casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)

# Character level
# character_input=Input(shape=(None,52,),name='char_input')
# embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5)(embed_char_out)
# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)



# output = concatenate([words, char])
# output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)

# output = TimeDistributed(Dense(len(label2id), activation='softmax'))(output)

# model = Model(inputs=[words_input, character_input], outputs=[output])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
# model.summary()


In [87]:
model.fit([X_train_word[:1000],X_train_char[:1000]],
          y_train[:1000],
          epochs=5,
          validation_data=([X_train_word[1000:1100],X_train_char[1000:1100]], y_train[1000:1100]))

Epoch 1/5
32/32 [==============================] - 14s 448ms/step - loss: 0.0719 - accuracy: 0.9083 - val_loss: 0.0602 - val_accuracy: 0.9443
Epoch 2/5
32/32 [==============================] - 13s 395ms/step - loss: 0.0628 - accuracy: 0.9083 - val_loss: 0.0506 - val_accuracy: 0.9443
Epoch 3/5
32/32 [==============================] - 12s 391ms/step - loss: 0.0580 - accuracy: 0.9078 - val_loss: 0.0477 - val_accuracy: 0.9443
Epoch 4/5
32/32 [==============================] - 13s 412ms/step - loss: 0.0540 - accuracy: 0.9066 - val_loss: 0.0469 - val_accuracy: 0.9453
Epoch 5/5
32/32 [==============================] - 12s 384ms/step - loss: 0.0502 - accuracy: 0.9068 - val_loss: 0.0470 - val_accuracy: 0.9467


'EU rejects German call to boycott British lamb .'

# References

https://keras.io/examples/nlp/ner_transformers/